# **Assignment 4**
# **Tasks**
1. Download the Dataset:- Dataset
2. Import required library
3. Read dataset and do pre-processing
4. Create Model
5. Add Layers (LSTM, Dense-(Hidden Layers), Output)
6. Compile the Model
7. Fit the Model
8. Save The Model
9. Test The Model

Load The Data Set

In [1]:
dataset_location = '/content/spam.csv'

Import the library

In [2]:
import pandas as pd
import nltk
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.translate.ribes_score import word_rank_alignment
from numpy.lib.shape_base import split
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import LSTM,Dense,Dropout,Input,Embedding,Activation,Flatten
from keras.models import Model
import nltk

Read dataset and do preprocessing

In [3]:
data = pd.read_csv(dataset_location,encoding = "ISO-8859-1")


In [4]:
data.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis = 1,inplace = True)
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
nltk.download('stopwords',quiet=True)
nltk.download('all',quiet=True)

True

In [6]:
ps = PorterStemmer()
input = []

In [7]:
for i in range(0,5572):
  v2 = data['v2'][i]

  #removing punctuation
  v2 = re.sub('[^a-zA-Z]',' ',v2)

  #converting to lower case
  v2 = v2.lower()

  #splitting the sentence
  v2 = v2.split()

  #removing the stopwords and stemming
  v2 = [ps.stem(word) for word in v2 if not word in set(stopwords.words('english'))]

  v2 = ' '.join(v2)

  input.append(v2)

In [8]:
#creating document term matrix
cv = CountVectorizer(max_features=2000)
x = cv.fit_transform(input).toarray()
x.shape

(5572, 2000)

In [9]:
le = preprocessing.LabelEncoder()

data['v1'] = le.fit_transform(data['v1'])
data['v1'].unique()


array([0, 1])

In [10]:
y = data['v1'].values


In [11]:
y = y.reshape(-1,1)

In [12]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.4)

Model building - Adding layers, Compiling model and saving model


In [13]:
model = Sequential()

In [14]:
model.add(Dense(1565,activation = "relu"))
model.add(Dense(3000,activation = "relu"))
model.add(Dense(1,activation = "sigmoid"))
model.add(Flatten())

In [15]:
model.compile(optimizer = "adam",loss = "binary_crossentropy", metrics = ["accuracy"])

In [16]:
model.fit(x_train,y_train,epochs = 15)

Epoch 1/15
105/105 [==============================] - 5s 41ms/step - loss: 0.1452 - accuracy: 0.9599
Epoch 2/15
105/105 [==============================] - 5s 47ms/step - loss: 0.0193 - accuracy: 0.9949
Epoch 3/15
105/105 [==============================] - 4s 42ms/step - loss: 0.0053 - accuracy: 0.9982
Epoch 4/15
105/105 [==============================] - 4s 42ms/step - loss: 0.0029 - accuracy: 0.9994
Epoch 5/15
105/105 [==============================] - 4s 43ms/step - loss: 0.0029 - accuracy: 0.9991
Epoch 6/15
105/105 [==============================] - 5s 43ms/step - loss: 0.0025 - accuracy: 0.9994
Epoch 7/15
105/105 [==============================] - 4s 43ms/step - loss: 0.0025 - accuracy: 0.9991
Epoch 8/15
105/105 [==============================] - 4s 42ms/step - loss: 0.0025 - accuracy: 0.9991
Epoch 9/15
105/105 [==============================] - 4s 42ms/step - loss: 0.0023 - accuracy: 0.9991
Epoch 10/15
105/105 [==============================] - 4s 42ms/step - loss: 0.0023 - accura

In [17]:
model.save("spam-message-classifier.h5")

Testing the model

In [18]:
ham = "im donee. come pick me up"
spam = "WINNER$$$$ SMS REPLY 'WIN'"
message = re.sub('[^a-zA-Z]',' ',spam)
message

'WINNER     SMS REPLY  WIN '

Testing with spam message

In [19]:
message = message.split()
message = [ps.stem(word) for word in message if not word in set(stopwords.words('english')) ]
message = ' '.join(message)


In [20]:
message1 = cv.transform([message])
message1


<1x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [21]:
TruePredction = model.predict(message1.astype(float))

1/1 [==============================] - 0s 109ms/step


In [22]:
TruePredction > 0.5

array([[ True]])

Testing with normal message

In [23]:
msg = re.sub('[^a-zA-Z]',' ',ham)
msg

'im donee  come pick me up'

In [24]:
msg = msg.split()
msg = [ps.stem(word) for word in msg if not word in set(stopwords.words('english'))]
msg = ' '.join(msg)

In [25]:
msg

'im done come pick'

In [26]:
cv.transform([msg])

<1x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [27]:
FalsePredection = model.predict(cv.transform([msg]))

1/1 [==============================] - 0s 68ms/step


In [28]:
FalsePredection > 0.5


array([[False]])